In [ ]:
from glob import glob
import pandas as pdlib
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import glob
from time import time, ctime
from datetime import datetime

In [ ]:
#Global Constants

dbloc ='<SQLite DB Loc>'

dateTimeObj = datetime.now()
dateObj = dateTimeObj.date()
curr_date_str = str (dateObj.year)+"_" + str(dateObj.month) +"_" + str(dateObj.day)
print(curr_date_str)

#Finviz Ticker Master File 
finviz_ticker_master_file='<finviz ticker file>'

#ARK Fund File locations
arkk = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_INNOVATION_ETF_ARKK_HOLDINGS.csv'
arkq = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_AUTONOMOUS_TECHNOLOGY_&_ROBOTICS_ETF_ARKQ_HOLDINGS.csv'
arkw = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_NEXT_GENERATION_INTERNET_ETF_ARKW_HOLDINGS.csv'
arkg = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_GENOMIC_REVOLUTION_MULTISECTOR_ETF_ARKG_HOLDINGS.csv'
arkf = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_FINTECH_INNOVATION_ETF_ARKF_HOLDINGS.csv'
arkx = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_SPACE_EXPLORATION_&_INNOVATION_ETF_ARKX_HOLDINGS.csv'
prnt = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/THE_3D_PRINTING_ETF_PRNT_HOLDINGS.csv'
izrl = 'https://ark-funds.com/wp-content/fundsiteliterature/csv/ARK_ISRAEL_INNOVATIVE_TECHNOLOGY_ETF_IZRL_HOLDINGS.csv'

#ARK CSV Folder
ark_csv_base_folder='<csv download folder>'
finviz_csv_base_folder='<finviz csvs>'
finviz_all_ticker_file='<finviz all tickers>'

In [ ]:
# Common Functions
def get_pricing_history(ticker_array) :
    try :
        df_ark_funds_pricing=pdlib.DataFrame()
        for symbol in ticker_array : 
            stock = yf.Ticker(symbol)
            curr_df = stock.history(period="max")
            curr_df['ticker'] = symbol
            df_ark_funds_pricing = df_ark_funds_pricing.append(curr_df)    
        return df_ark_funds_pricing
    except :
        print('Error  ' + symbol)

def get_pricing_chart(ticker_array) :
    try:
        df_fund_pricing=get_pricing_history(ticker_array)
        for ticker in ticker_array :
            df_fund_pricing.loc[df_fund_pricing['ticker'] == ticker]['Close'].plot(label=ticker,figsize=(16,8),title='Closing Price')
        plt.legend()
    except :
        print('Error  ' + symbol)
        
def load_all_ark_csvs(path) :
    dfs = []
    for filename in glob.glob(path):
        print('loading..'+ filename)
        dfs.append(pd.read_csv(filename))
    big_frame = pdlib.concat(dfs, ignore_index=True)
    return big_frame.dropna()
    
path = '<ark_csv_base_folder>/sources/ark/ark-csvs/*'
ark_data=(load_all_ark_csvs(path))
print(ark_data)
        
#ticker_array = {"MSFT","IBM","TSLA"}        
#get_pricing_chart(ticker_array)

In [ ]:
#Question 1 - How ARK fund is distributed by market value
pdlib.options.display.float_format = '{:,.2f}'.format
ark_data.groupby(['fund']).sum()

In [ ]:
#Question 2 - How many distinct stocks are there are across the ARK portfolio
ark_ticker_list=ark_data.ticker.unique()
print('Total Distinct Instruments [Stocks, ETFs..] '+ str(len(ark_ticker_list)))

In [ ]:
#Question 3 - How many are listed in US market vs. International
df_all_tickers=pd.read_csv(finviz_ticker_master_file)
df_all_tickers.head()
stocks_with_info_list =[]
stocks_without_info_list =[]

for symbol in ark_ticker_list :
    symbol_count=df_all_tickers['Ticker'].str.contains(symbol).sum()
    if symbol_count>0:
        stocks_with_info_list.append(symbol)
    else :
        stocks_without_info_list.append(symbol) 
        
print('Listed in US Market  ' + str(len(stocks_with_info_list)))
print('Not Listed in US Market ' + str(len(stocks_without_info_list)))


In [ ]:
#Question 4 - How ARK funds compared to one another 
ark_funds_array={'ARKK','ARKQ','ARKW','ARKG','ARKF','ARKX','PRNT','IZRL'}
df_ark_funds=df_all_tickers.loc[df_all_tickers['Ticker'].isin(ark_funds_array)]

Funds = df_ark_funds['Ticker']
Price =  df_ark_funds['Price']

xAxis = [i + 0.5 for i, _ in enumerate(Funds)]
  
plt.bar(xAxis, Price, color='teal')
plt.title('Funds Vs Price', fontsize=14)
plt.xlabel('Funds', fontsize=14)
plt.ylabel('Price', fontsize=14)
plt.xticks([i + 0.5 for i, _ in enumerate(Funds)], Funds)
plt.show()

In [ ]:
#Question 5 - How ARK Funds pricing performance look over the years
get_pricing_chart(ark_funds_array)


In [ ]:
#Question 6 - How ARK Funds Underlying instruments are performing

def p2f(x):
    try :
        x = float(x.strip('%'))
        return(x)
    except :
        return 0
all_tickers_csv_file=finviz_csv_base_folder+finviz_all_ticker_file
df_ark_all_tickers = pdlib.read_csv(all_tickers_csv_file,converters={'20-Day Simple Moving Average':p2f,'50-Day Simple Moving Average':p2f,'Profit Margin':p2f })
df_ark_all_tickers = df_ark_all_tickers[df_ark_all_tickers['Ticker'].isin(stocks_with_info_list)]

df_temp = df_ark_all_tickers.loc[ 
    #(df_ark_all_tickers['Sector'] =='Technology') &
    (df_ark_all_tickers['EPS (ttm)'] > 0) &
    (df_ark_all_tickers['Current Ratio'] > 1.5) &  
    (df_ark_all_tickers['20-Day Simple Moving Average'] > 0 ) &
    (df_ark_all_tickers['Relative Strength Index (14)'] > 60 ) 
    ]

In [ ]:
df_temp.shape

In [ ]:
#Question 6.1 - Top 10 Instruments by price
pd.set_option("display.max.columns", None)
print('Top 10 Instruments by Price: ')
(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Price'], ascending=False).head(10)
#print(df_calc)
#ticker_array=df_calc['Ticker']
#get_pricing_chart(ticker_array)

In [ ]:
#Question 6.2 - Bottom 10 Instruments by price
print('Bottom 10 Instruments : ')
#df_calc = (df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Price'], ascending=False).tail(10)
(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Price'], ascending=False).tail(10)
#print(df_calc)
#ticker_array=df_calc['Ticker']
#get_pricing_chart(ticker_array)

In [ ]:
#Question 7.1 - What are the Top 10 most active Instruments 
print('Most Active 10 Instruments : ')
#df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Volume']]).sort_values(by=['Volume'], ascending=False).head(10)
(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Volume']]).sort_values(by=['Volume'], ascending=False).tail(10)
#print(df_calc)
#ticker_array=df_calc['Ticker']
#get_pricing_chart(ticker_array)

In [ ]:
#Question 7.2 - What are the  10 least active Instruments 
print('Least Active 10 Instruments : ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Volume']]).sort_values(by=['Volume'], ascending=False).tail(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 8.1 - What are the 10 most profitable Instruments 
print('Most Profitable 10 Instruments : ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Profit Margin'], ascending=False).head(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 8.2 - What are the 10 most profitable Instruments 
print('Least Profitable 10 Instruments : ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Profit Margin'], ascending=False).tail(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 9 - How does the sector-wise investments looks like in Healthcare

df_ark_all_tickers = df_ark_all_tickers.loc[ (df_ark_all_tickers['Sector'] =='Healthcare')]
    
#print('Top 10 Instruments in Healthcare by Price : ')
(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Price'], ascending=False).tail(10)
#print(df_calc)
#ticker_array=df_calc['Ticker']
#get_pricing_chart(ticker_array)

In [ ]:
#Question 9.1
print('Bottom 10 Instruments in Healthcare by Price: ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Price'], ascending=False).tail(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 9.2
print('Most Active 10 Instruments in Healthcare : ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Volume']]).sort_values(by=['Volume'], ascending=False).head(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 9.3
print('Least Active 10 Instruments in Healthcare : ')
(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Volume']]).sort_values(by=['Volume'], ascending=False).tail(10)
#print(df_calc)
#ticker_array=df_calc['Ticker']
#get_pricing_chart(ticker_array)

In [ ]:
#Question 9.4
print('Most Profitable 10 Instruments in Healthcare: ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Profit Margin'], ascending=False).head(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)

In [ ]:
#Question 9.5
print('Least Profitable 10 Instruments in Healthcare: ')
df_calc=(df_ark_all_tickers[['Ticker','Sector','Industry','Company','Market Cap','Price','P/E','PEG','P/S','Profit Margin']]).sort_values(by=['Profit Margin'], ascending=False).tail(10)
print(df_calc)
ticker_array=df_calc['Ticker']
get_pricing_chart(ticker_array)


In [ ]:
df_ark_all_tickers.corr(method ='pearson')

In [ ]:
df_ark_all_tickers.corr(method ='kendall')